In [1]:
# IMPORTS
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, make_scorer, f1_score, precision_score, recall_score
from imblearn.over_sampling import SMOTE

import seaborn as sns
import matplotlib.pyplot as plt
import time

In [2]:
# 1. CHARGEMENT DES DONNÉES
print("=== CHARGEMENT DES DONNÉES ===")

try:
    # https://www.kaggle.com/datasets/joniarroba/noshowappointments/data
    path = "no-show.csv"
    df = pd.read_csv(path)
    # rename car y a des titres mal orthographiés
    df = df.rename(columns={'Handcap': 'Handicap', 'Hipertension': 'Hypertension'})


except:
    print("Erreur lors du chargement des données")
    
print( "=" * 80)
print("\n5 Premières lignes:")
print(df.head())
print( "=" * 80)

=== CHARGEMENT DES DONNÉES ===

5 Premières lignes:
      PatientId  AppointmentID Gender          ScheduledDay  \
0  2.987250e+13        5642903      F  2016-04-29T18:38:08Z   
1  5.589978e+14        5642503      M  2016-04-29T16:08:27Z   
2  4.262962e+12        5642549      F  2016-04-29T16:19:04Z   
3  8.679512e+11        5642828      F  2016-04-29T17:29:31Z   
4  8.841186e+12        5642494      F  2016-04-29T16:07:23Z   

         AppointmentDay  Age      Neighbourhood  Scholarship  Hypertension  \
0  2016-04-29T00:00:00Z   62    JARDIM DA PENHA            0             1   
1  2016-04-29T00:00:00Z   56    JARDIM DA PENHA            0             0   
2  2016-04-29T00:00:00Z   62      MATA DA PRAIA            0             0   
3  2016-04-29T00:00:00Z    8  PONTAL DE CAMBURI            0             0   
4  2016-04-29T00:00:00Z   56    JARDIM DA PENHA            0             1   

   Diabetes  Alcoholism  Handicap  SMS_received No-show  
0         0           0         0         

## STRATEGIE : 2° EXPLORATION RAPIDE

Mettre les metrics et les moyennes générales.

In [3]:
# 2. EXPLORATION RAPIDE
print("\n=== EXPLORATION RAPIDE DES DONNÉES ===")
# Afficher la forme du dataset et les colonnes
print(f"Forme du dataset: {df.shape}")
print("\n")
print(f"Colonnes: {list(df.columns)}")
print("=" * 80)

# Explorer les types de données et les valeurs manquantes
print("\n")
print("\n=> Types de données et valeurs manquantes:\n")
print(df.info())
print("=" * 80)

# Afficher les statistiques descriptives
print("\n")
print("=> Valeurs manquantes:\n")
print(df.isnull().sum())
print("\n=> PAS DE CELLULES VIDES !!")
print("=" * 80)

print("\n=== METRICS GENERALES ===")
# Calcul du taux de No-show
total_rows = len(df)
no_show_count = (df['No-show'] == 'Yes').sum()
no_show_rate = (no_show_count / total_rows) * 100
print(f"\n=> Taux de No Show TOTAL : {no_show_count}/{total_rows} = {no_show_rate:.2f}%")

# Nombre de femmes
femmes_count = (df['Gender'] == 'F').sum()
femmes_rate = (femmes_count / total_rows) * 100
print(f"=> Nombre de Femmes : {femmes_count}/{total_rows} = {femmes_rate:.2f}%")

# Nombre avec Scholarship
scholarship_count = (df['Scholarship'] == 1).sum()
scholarship_rate = (scholarship_count / total_rows) * 100
print(f"=> Population Ayant un certain niveau d'études Scholarship : {scholarship_count}/{total_rows} = {scholarship_rate:.2f}%")

# Nombre avec Hypertension
hypertension_count = (df['Hypertension'] == 1).sum()
hypertension_rate = (hypertension_count / total_rows) * 100
print(f"=> population Ayant de l'hypertension : {hypertension_count}/{total_rows} = {hypertension_rate:.2f}%")

# Nombre avec SMS reçu
sms_count = (df['SMS_received'] == 1).sum()
sms_rate = (sms_count / total_rows) * 100
print(f"=> population Ayant reçu un SMS : {sms_count}/{total_rows} = {sms_rate:.2f}%")

# Nombre avec Handicap
handicap_count = (df['Handicap'] > 0).sum()
handicap_rate = (handicap_count / total_rows) * 100
print(f"=> population Ayant un handicap : {handicap_count}/{total_rows} = {handicap_rate:.2f}%")

# Nombre avec Alcoholism
alcoholism_count = (df['Alcoholism'] == 1).sum()
alcoholism_rate = (alcoholism_count / total_rows) * 100
print(f"=> population Ayant un alcoolisme : {alcoholism_count}/{total_rows} = {alcoholism_rate:.2f}%")

# Nombre avec Diabetes
diabetes_count = (df['Diabetes'] == 1).sum()
diabetes_rate = (diabetes_count / total_rows) * 100
print(f"=> population Ayant un diabète : {diabetes_count}/{total_rows} = {diabetes_rate:.2f}%")


=== EXPLORATION RAPIDE DES DONNÉES ===
Forme du dataset: (110527, 14)


Colonnes: ['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay', 'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hypertension', 'Diabetes', 'Alcoholism', 'Handicap', 'SMS_received', 'No-show']



=> Types de données et valeurs manquantes:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hypertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcohol

## STRATEGIE / 3° Pré - features engineering


### Colonnes du dataset à garder : 

- Gender => M or F	
- Age => int
- Neighbourhood => string	
- Scholarship	
- Hypertension	
- Diabetes	
- Alcoholism	
- Handicap	
- SMS_received	
- No-show

### Nouvelles colonnes à créer :

- "DaysUntilAppointement" => calcul en jours (approx) de AppointmentDay - ScheduledDay


### Colonnes du dataset à virer :

- PatientId	
- AppointmentID	
- ScheduledDay (après avoir créé DaysUntilAppointement)	
- AppointmentDay (après avoir créé DaysUntilAppointement)

=> Le label est "No-show"



### Voir si pas de cellules abbérantes

- No show => Yes/No only
- Age : pas de négatif et pas de + de 115 ans et que des integers 
- Scholarship	/ Hypertension / Diabetes / Alcoholism	/ Handicap	/ SMS_received / No-show : que de 0 ou 1
- Gender: ue des M ou F           
- Neighbourhood doit être une string 
- AppointmentDay et ScheduledDay doivent être du type : 18/05/2016  09:18:03
- 'Handicap' ne contient que les valeurs [0 1 2 3 4]:

In [4]:
# Contrôle de Y
print("\n=== CONTRÔLE DE VALEURS ABERRANTES POUR LA TARGET NO-SHOW ===")

# Vérification des valeurs uniques dans No-show
unique_values = df['No-show'].unique()
print(f"Valeurs uniques dans 'No-show': {unique_values}")
print(" Contrôle OK : La colonne 'No-show' ne contient que 'Yes' et 'No'")


=== CONTRÔLE DE VALEURS ABERRANTES POUR LA TARGET NO-SHOW ===
Valeurs uniques dans 'No-show': ['No' 'Yes']
 Contrôle OK : La colonne 'No-show' ne contient que 'Yes' et 'No'


In [5]:
print("\n=== CONTRÔLE DE VALEURS ABERRANTES DE X ===")
# on degage les colonnes PatientId et AppointmentID
relevantColumns = ['No-show', 'Scholarship', 'Gender', 'Age', 'Hypertension', 'Diabetes', 'Alcoholism', 'Handicap', 'SMS_received', 'ScheduledDay', 'AppointmentDay', 'Neighbourhood']
preSelectedDf = df[relevantColumns].copy()

# Contrôle des colonnes numériques binaires (0/1)
print("\n=> Contrôle Scholarship', 'Hypertension', 'Diabetes', 'Alcoholism', 'SMS_received:")
binary_columns = ['Scholarship', 'Hypertension', 'Diabetes', 'Alcoholism', 'SMS_received']
for col in binary_columns:
    unique_vals = preSelectedDf[col].unique()
    if set(unique_vals).issubset({0, 1}):
        print(f" Contrôle OK : La colonne '{col}' ne contient que des valeurs binaires (0/1)")
    else:
        print(f" Problème détecté : La colonne '{col}' contient des valeurs autres que 0 et 1: {unique_vals}")  

# Contrôle de la colonne 'Handicap'
print("\n=> Contrôle Handicap:")
handicap_vals = preSelectedDf['Handicap'].unique()
expected_handicap = {0, 1, 2, 3, 4}
if set(handicap_vals).issubset(expected_handicap):
    print(f" Contrôle OK : La colonne 'Handicap' contient uniquement des valeurs attendues: {sorted(handicap_vals)}")
else:
    unexpected = set(handicap_vals) - expected_handicap
    print(f" Problème détecté : La colonne 'Handicap' contient des valeurs inattendues: {unexpected}")

# Contrôle de la colonne 'Gender'
print("\n=> Contrôle Gender:")
gender_vals = preSelectedDf['Gender'].unique()
if set(gender_vals) == {'M', 'F'}:
    print(f" Contrôle OK : La colonne 'Gender' ne contient que 'M' et 'F'")
else:
    print(f" Problème détecté : La colonne 'Gender' contient des valeurs autres que M/F: {gender_vals}")

# Contrôle de la colonne 'Neighbourhood'
print("\n=> Contrôle Neighbourhood:")

# Afficher les valeurs uniques
unique_neighbourhoods = preSelectedDf['Neighbourhood'].unique()
nb_unique = preSelectedDf['Neighbourhood'].nunique()
print(f"  Nombre de quartiers uniques: {nb_unique}")
print(f"  Echantillon de 20 quartiers: {sorted(unique_neighbourhoods)[:20]}")

neighbourhood_type = preSelectedDf['Neighbourhood'].dtype
if neighbourhood_type == 'object' or neighbourhood_type == 'string':
    nb_unique = preSelectedDf['Neighbourhood'].nunique()
    print(f" Contrôle OK : La colonne 'Neighbourhood' est de type string avec {nb_unique} quartiers uniques")
else:
    print(f" Problème détecté : La colonne 'Neighbourhood' n'est pas de type string: {neighbourhood_type}")

# Contrôle des colonnes 'AppointmentDay' et 'ScheduledDay'
print("\n=> Contrôle des dates:")
import re
# Pattern pour le format ISO 8601: YYYY-MM-DDTHH:MM:SSZ
date_pattern = r'^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z$'

for col in ['AppointmentDay', 'ScheduledDay']:
    # Afficher un échantillon
    sample_value = str(preSelectedDf[col].iloc[0])
    print(f"  Échantillon de '{col}': {sample_value}")
    
    # Vérifier le type
    if preSelectedDf[col].dtype == 'object':
        print(f"  '{col}' est de type string/object")
        
        # Vérifier le format sur toutes les valeurs
        format_ok = preSelectedDf[col].astype(str).str.match(date_pattern).all()
        
        if format_ok:
            print(f" Toutes les valeurs de '{col}' respectent le format ISO 8601 (YYYY-MM-DDTHH:MM:SSZ)")
        else:
            # Compter combien ne respectent pas le format
            format_issues = (~preSelectedDf[col].astype(str).str.match(date_pattern)).sum()
            print(f" Problème : {format_issues} valeurs ne respectent pas le format ISO 8601")
            # Afficher quelques exemples de valeurs problématiques
            invalid_samples = preSelectedDf[~preSelectedDf[col].astype(str).str.match(date_pattern)][col].head(3)
            print(f"  Exemples de valeurs invalides: {list(invalid_samples)}")
    else:
        print(f"  '{col}' est de type: {preSelectedDf[col].dtype}")

# Contrôle 'Age': pas de négatif et pas de + de 115 ans et que des integers
age_min = preSelectedDf['Age'].min()
age_max = preSelectedDf['Age'].max()
age_dtype = preSelectedDf['Age'].dtype
has_negative = (preSelectedDf['Age'] < 0).any()
has_too_old = (preSelectedDf['Age'] > 115).any()

print(f"\n=> Contrôle de la colonne 'Age':")
print(f"  Type: {age_dtype}, Min: {age_min}, Max: {age_max}")

if has_negative:
    count_negative = (preSelectedDf['Age'] < 0).sum()
    print(f" Problème : {count_negative} valeurs négatives détectées dans 'Age'")
    print("\n=== NETTOYAGE DES DONNÉES ABERRANTES ===")
    # Suppression des lignes avec âge négatif
    nb_rows_before = len(preSelectedDf)
    preSelectedDf = preSelectedDf[preSelectedDf['Age'] >= 0]
    nb_rows_after = len(preSelectedDf)
    nb_deleted = nb_rows_before - nb_rows_after

    if nb_deleted > 0:
        print(f" {nb_deleted} ligne(s) avec Age négatif supprimée(s)")
        print(f"  Nombre de lignes avant: {nb_rows_before}")
        print(f"  Nombre de lignes après: {nb_rows_after}")

else:
    print(f" Pas de valeurs négatives dans 'Age'")

if has_too_old:
    count_too_old = (preSelectedDf['Age'] > 115).sum()
    print(f" Problème : {count_too_old} valeurs > 115 ans détectées dans 'Age'")
else:
    print(f" Pas de valeurs > 115 ans dans 'Age'")


=== CONTRÔLE DE VALEURS ABERRANTES DE X ===

=> Contrôle Scholarship', 'Hypertension', 'Diabetes', 'Alcoholism', 'SMS_received:
 Contrôle OK : La colonne 'Scholarship' ne contient que des valeurs binaires (0/1)
 Contrôle OK : La colonne 'Hypertension' ne contient que des valeurs binaires (0/1)
 Contrôle OK : La colonne 'Diabetes' ne contient que des valeurs binaires (0/1)
 Contrôle OK : La colonne 'Alcoholism' ne contient que des valeurs binaires (0/1)
 Contrôle OK : La colonne 'SMS_received' ne contient que des valeurs binaires (0/1)

=> Contrôle Handicap:
 Contrôle OK : La colonne 'Handicap' contient uniquement des valeurs attendues: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]

=> Contrôle Gender:
 Contrôle OK : La colonne 'Gender' ne contient que 'M' et 'F'

=> Contrôle Neighbourhood:
  Nombre de quartiers uniques: 81
  Echantillon de 20 quartiers: ['AEROPORTO', 'ANDORINHAS', 'ANTÔNIO HONÓRIO', 'ARIOVALDO FAVALESSA', 'BARRO VERMELHO', 'BELA VISTA', 'BENTO FERR

In [6]:
print("\n=== CRÉATION DE LA NOUVELLE COLONNE 'DaysUntilAppointement' ===")
# Conversion des colonnes de dates en datetime
preSelectedDf['AppointmentDay'] = pd.to_datetime(preSelectedDf['AppointmentDay'])
preSelectedDf['ScheduledDay'] = pd.to_datetime(preSelectedDf['ScheduledDay'])

# il y a un biais dans les dates car on dans 30% des cas on a ScheduledDay > AppointmentDay, ce qui est impossible.
# et dans la plupart des cas, on a des rdv ayant lieu le même jour que la réservation : 
# 2016-04-29 18:38:08+00:00 > Appointment: 2016-04-29 00:00:00+00:00
# ceci s'explique que les heures de la colonne AppointmentDay sont reset à 00h00, et celles de : ScheduledDay ne le sont pas
# Il faut donc remettre ScheduledDay et AppointmentDayà niveau, et transformer tous ses datetimes en 00:00:00
print("\n=> Reset à zéro heures pour les 2 colonnes:")
preSelectedDf['ScheduledDay'] = preSelectedDf['ScheduledDay'].dt.normalize()
preSelectedDf['AppointmentDay'] = preSelectedDf['AppointmentDay'].dt.normalize()

# Contrôle des dates aberrantes (AppointmentDay < ScheduledDay)
print("\n=> Contrôle des AppointmentDay < ScheduledDay détectées (ANOMALIES):")
dates_aberrantes = preSelectedDf['AppointmentDay'] < preSelectedDf['ScheduledDay']
nb_dates_aberrantes = dates_aberrantes.sum()

if nb_dates_aberrantes > 0:
    print(f" Problème : {nb_dates_aberrantes} ligne(s) où AppointmentDay < ScheduledDay détectées")
    print(f"  Ces lignes seront supprimées")
    
    # Afficher quelques exemples
    exemples = preSelectedDf[dates_aberrantes][['ScheduledDay', 'AppointmentDay']].head(3)
    print(f"\n  Exemples de dates aberrantes:")
    for idx, row in exemples.iterrows():
        print(f"    Scheduled: {row['ScheduledDay']} > Appointment: {row['AppointmentDay']}")
    
    # Suppression des lignes aberrantes
    nb_rows_before = len(preSelectedDf)
    preSelectedDf = preSelectedDf[~dates_aberrantes]
    nb_rows_after = len(preSelectedDf)
    print(f"\n {nb_rows_before - nb_rows_after} ligne(s) supprimée(s)")
    print(f"  Lignes restantes: {nb_rows_after}")
else:
    print(f" Toutes les dates sont cohérentes (AppointmentDay >= ScheduledDay)")

# Calcul de la différence en jours (arrondi au supérieur)
preSelectedDf['DaysUntilAppointement'] = (preSelectedDf['AppointmentDay'] - preSelectedDf['ScheduledDay']).dt.total_seconds() / (24 * 3600)
preSelectedDf['DaysUntilAppointement'] = np.ceil(preSelectedDf['DaysUntilAppointement']).astype(int)

print(f" Colonne 'DaysUntilAppointement' créée avec succès")
print(f"  Min: {preSelectedDf['DaysUntilAppointement'].min()} jours")
print(f"  Max: {preSelectedDf['DaysUntilAppointement'].max()} jours")
print(f"  Moyenne: {preSelectedDf['DaysUntilAppointement'].mean():.2f} jours")

# On supprime les colonnes de dates maintenant qu'on a extrait l'information
print("\n=== SUPPRESSION DES COLONNES 'AppointmentDay', 'ScheduledDay' ===")
preSelectedDf = preSelectedDf.drop(columns=['AppointmentDay', 'ScheduledDay'])
print(f" Colonnes 'AppointmentDay' et 'ScheduledDay' supprimées")
print(f"  Colonnes restantes: {list(preSelectedDf.columns)}")
print("=" * 80)


=== CRÉATION DE LA NOUVELLE COLONNE 'DaysUntilAppointement' ===



=> Reset à zéro heures pour les 2 colonnes:

=> Contrôle des AppointmentDay < ScheduledDay détectées (ANOMALIES):
 Problème : 5 ligne(s) où AppointmentDay < ScheduledDay détectées
  Ces lignes seront supprimées

  Exemples de dates aberrantes:
    Scheduled: 2016-05-10 00:00:00+00:00 > Appointment: 2016-05-09 00:00:00+00:00
    Scheduled: 2016-05-18 00:00:00+00:00 > Appointment: 2016-05-17 00:00:00+00:00
    Scheduled: 2016-05-05 00:00:00+00:00 > Appointment: 2016-05-04 00:00:00+00:00

 5 ligne(s) supprimée(s)
  Lignes restantes: 110521
 Colonne 'DaysUntilAppointement' créée avec succès
  Min: 0 jours
  Max: 179 jours
  Moyenne: 10.18 jours

=== SUPPRESSION DES COLONNES 'AppointmentDay', 'ScheduledDay' ===
 Colonnes 'AppointmentDay' et 'ScheduledDay' supprimées
  Colonnes restantes: ['No-show', 'Scholarship', 'Gender', 'Age', 'Hypertension', 'Diabetes', 'Alcoholism', 'Handicap', 'SMS_received', 'Neighbourhood', 'DaysUntilAppointement']


In [7]:
# Création de la colonne nb_pathologies
print("\n=== CRÉATION DE LA COLONNE 'nb_pathologies' ===")
preSelectedDf['nb_pathologies'] = (preSelectedDf['Hypertension'] + 
                                    preSelectedDf['Diabetes'] + 
                                    preSelectedDf['Alcoholism'])
print(f" Colonne 'nb_pathologies' créée avec succès")
print(f"  Valeurs possibles: {sorted(preSelectedDf['nb_pathologies'].unique())}")
print("=" * 80)


=== CRÉATION DE LA COLONNE 'nb_pathologies' ===
 Colonne 'nb_pathologies' créée avec succès
  Valeurs possibles: [np.int64(0), np.int64(1), np.int64(2), np.int64(3)]


In [8]:
# SUPPRESSION DE LA COLONNE SMS_received car valeur biaisée
print("\n=== SUPPRESSION DE LA COLONNE 'SMS_received' ===")
preSelectedDf = preSelectedDf.drop(columns=['SMS_received'])
print(f" Colonne 'SMS_received' supprimée")
print(f"  Colonnes restantes: {list(preSelectedDf.columns)}")
print("=" * 80)


=== SUPPRESSION DE LA COLONNE 'SMS_received' ===
 Colonne 'SMS_received' supprimée
  Colonnes restantes: ['No-show', 'Scholarship', 'Gender', 'Age', 'Hypertension', 'Diabetes', 'Alcoholism', 'Handicap', 'Neighbourhood', 'DaysUntilAppointement', 'nb_pathologies']


### (NEW!) Transformation de Neighbourhood No-show en numériques

- On utilise LabelEncoder pour transformer les noms de quartiers en entiers
- On transforme les valeurs Yes/No en 1/0

In [9]:
print("\n=== ENCODAGE DE 'Neighbourhood' EN INTEGERS ===")
# Créer un LabelEncoder pour Neighbourhood
neighbourhood_encoder = LabelEncoder()
preSelectedDf['Neighbourhood'] = neighbourhood_encoder.fit_transform(preSelectedDf['Neighbourhood'])

print(f" Colonne 'Neighbourhood' encodée en integers")
print(f"  Nombre de quartiers uniques: {preSelectedDf['Neighbourhood'].nunique()}")
print(f"  Valeurs min/max: {preSelectedDf['Neighbourhood'].min()} / {preSelectedDf['Neighbourhood'].max()}")
print(f"  Type: {preSelectedDf['Neighbourhood'].dtype}")

# Afficher quelques exemples de mapping
print("\n  Exemples de mapping (5 premiers quartiers):")
for i, neighbourhood in enumerate(neighbourhood_encoder.classes_[:5]):
    print(f"    '{neighbourhood}' -> {i}")

print("=" * 80)


=== ENCODAGE DE 'Neighbourhood' EN INTEGERS ===
 Colonne 'Neighbourhood' encodée en integers
  Nombre de quartiers uniques: 81
  Valeurs min/max: 0 / 80
  Type: int64

  Exemples de mapping (5 premiers quartiers):
    'AEROPORTO' -> 0
    'ANDORINHAS' -> 1
    'ANTÔNIO HONÓRIO' -> 2
    'ARIOVALDO FAVALESSA' -> 3
    'BARRO VERMELHO' -> 4


In [10]:
print("\n=== TRANSFORMATION DE 'No-show' EN 0/1 ===")
# Transformer Yes/No en 1/0
preSelectedDf['No-show'] = (preSelectedDf['No-show'] == 'Yes').astype(int)

print(f" Colonne 'No-show' transformée en 0/1")
print(f"  Valeurs uniques: {sorted(preSelectedDf['No-show'].unique())}")
print(f"  Type: {preSelectedDf['No-show'].dtype}")
print(f"\n  Mapping: No -> 0, Yes -> 1")
print(f"  Taux de no-show (1): {(preSelectedDf['No-show'] == 1).sum() / len(preSelectedDf) * 100:.2f}%")

print("=" * 80)


=== TRANSFORMATION DE 'No-show' EN 0/1 ===
 Colonne 'No-show' transformée en 0/1
  Valeurs uniques: [np.int64(0), np.int64(1)]
  Type: int64

  Mapping: No -> 0, Yes -> 1
  Taux de no-show (1): 20.19%


In [11]:
# Résumé final des données préparées
print("\n=== RÉSUMÉ DES DONNÉES PRÉPARÉES ===")
print(f"\nNombre total de lignes: {len(preSelectedDf)}")
print(f"\nColonnes finales ({len(preSelectedDf.columns)}):")
for col in preSelectedDf.columns:
    print(f"  - {col} ({preSelectedDf[col].dtype})")

print(f"\nAperçu des premières lignes:")
print(preSelectedDf.head())

print("\n" + "=" * 80)
print(" DONNÉES PRÊTES POUR L'ENTRAÎNEMENT")
print("=" * 80)


=== RÉSUMÉ DES DONNÉES PRÉPARÉES ===

Nombre total de lignes: 110521

Colonnes finales (11):
  - No-show (int64)
  - Scholarship (int64)
  - Gender (object)
  - Age (int64)
  - Hypertension (int64)
  - Diabetes (int64)
  - Alcoholism (int64)
  - Handicap (int64)
  - Neighbourhood (int64)
  - DaysUntilAppointement (int64)
  - nb_pathologies (int64)

Aperçu des premières lignes:
   No-show  Scholarship Gender  Age  Hypertension  Diabetes  Alcoholism  \
0        0            0      F   62             1         0           0   
1        0            0      M   56             0         0           0   
2        0            0      F   62             0         0           0   
3        0            0      F    8             0         0           0   
4        0            0      F   56             1         1           0   

   Handicap  Neighbourhood  DaysUntilAppointement  nb_pathologies  
0         0             39                      0               1  
1         0             39      

## (NEW!) ANALYSE DE CORRÉLATION AVEC LE LABEL

Avant de sélectionner les features, analysons leur corrélation avec `No-show` (label) pour identifier les features peu pertinentes.

In [12]:
# Analyse de corrélation entre les features et le label No-show
print("\n=== ANALYSE DE CORRÉLATION AVEC LE LABEL 'No-show' ===")
print("=" * 80)

# Encoder temporairement Gender pour le calcul de corrélation
df_for_corr = preSelectedDf.copy()
if df_for_corr['Gender'].dtype == 'object':
    df_for_corr['Gender'] = (df_for_corr['Gender'] == 'M').astype(int)  # M=1, F=0

# Calculer la corrélation de chaque feature avec No-show
correlations = df_for_corr.corr()['No-show'].sort_values(ascending=False)

print("\n Corrélation de chaque feature avec No-show (1 = absent, 0 = présent):")
print(f"{'' * 80}\n")

for feature, corr in correlations.items():
    if feature != 'No-show':
        # Déterminer la force de la corrélation
        abs_corr = abs(corr)
        if abs_corr > 0.10:
            strength = "FORTE"
        elif abs_corr > 0.05:
            strength = "MOYENNE"
        elif abs_corr > 0.02:
            strength = "FAIBLE"
        else:
            strength = "TRÈS FAIBLE"
        
        bar = '' * int(abs_corr * 100)
        print(f"  {feature:25s} : {corr:+.4f}  {bar}  {strength}")

print("\n INTERPRÉTATION:")
print("  - Corrélation > 0.10 ou < -0.10 : Feature PERTINENTE")
print("  - Corrélation entre -0.02 et +0.02 : Feature PEU PERTINENTE (peut être retirée)")
print("  - Corrélation positive : Plus la valeur augmente, plus le risque d'absence augmente")
print("  - Corrélation négative : Plus la valeur augmente, moins le risque d'absence")

print("\n" + "=" * 80)


=== ANALYSE DE CORRÉLATION AVEC LE LABEL 'No-show' ===

 Corrélation de chaque feature avec No-show (1 = absent, 0 = présent):


  DaysUntilAppointement     : +0.1863    FORTE
  Scholarship               : +0.0292    FAIBLE
  Alcoholism                : -0.0002    TRÈS FAIBLE
  Gender                    : -0.0041    TRÈS FAIBLE
  Handicap                  : -0.0063    TRÈS FAIBLE
  Neighbourhood             : -0.0090    TRÈS FAIBLE
  Diabetes                  : -0.0152    TRÈS FAIBLE
  nb_pathologies            : -0.0303    FAIBLE
  Hypertension              : -0.0357    FAIBLE
  Age                       : -0.0603    MOYENNE

 INTERPRÉTATION:
  - Corrélation > 0.10 ou < -0.10 : Feature PERTINENTE
  - Corrélation entre -0.02 et +0.02 : Feature PEU PERTINENTE (peut être retirée)
  - Corrélation positive : Plus la valeur augmente, plus le risque d'absence augmente
  - Corrélation négative : Plus la valeur augmente, moins le risque d'absence



## NETTOYAGE DES FEATURES

Basé sur l'analyse de corrélation, nous allons:
1. **Retirer les features redondantes** : `Diabetes`, `Hypertension`, `Alcoholism` (car incluses dans `nb_pathologies`)
2. **Retirer les features peu corrélées** : Celles avec corrélation absolue < 0.02 avec le label

In [13]:
# Identification et suppression des features non pertinentes
print("\n=== NETTOYAGE DES FEATURES ===")
print("=" * 80)

# 1. Features redondantes (incluses dans nb_pathologies)
redundant_features = ['Diabetes', 'Hypertension', 'Alcoholism']

# 2. Features peu corrélées avec le label (|corrélation| < 0.02)
low_corr_features = []
for feature, corr in correlations.items():
    if feature != 'No-show' and abs(corr) < 0.02:
        low_corr_features.append(feature)

# Combiner les deux listes
features_to_remove = list(set(redundant_features + low_corr_features))

print(f"\n Features à retirer ({len(features_to_remove)}):")
print(f"{'' * 80}")

if features_to_remove:
    for feat in sorted(features_to_remove):
        reason = ""
        if feat in redundant_features:
            reason = "REDONDANTE (incluse dans nb_pathologies)"
        if feat in low_corr_features:
            corr_val = correlations[feat]
            reason += f" + PEU CORRÉLÉE ({corr_val:+.4f})" if reason else f"PEU CORRÉLÉE ({corr_val:+.4f})"
        print(f"   {feat:25s} : {reason}")
    
    # Suppression des features
    print(f"\n⏳ Suppression des features non pertinentes...")
    features_before = list(preSelectedDf.columns)
    preSelectedDf = preSelectedDf.drop(columns=features_to_remove)
    features_after = list(preSelectedDf.columns)
    
    print(f" {len(features_to_remove)} feature(s) supprimée(s)")
    print(f"  Features avant: {len(features_before)} → Features après: {len(features_after)}")
else:
    print("  ℹ Aucune feature à retirer (toutes sont pertinentes)")

print(f"\n Features finales conservées ({len(preSelectedDf.columns) - 1} features + 1 label):")
print(f"{'' * 80}")
for col in sorted(preSelectedDf.columns):
    if col != 'No-show':
        corr_val = correlations[col] if col in correlations else 0
        print(f"   {col:25s} (corrélation: {corr_val:+.4f})")

print("\n" + "=" * 80)


=== NETTOYAGE DES FEATURES ===

 Features à retirer (6):

   Alcoholism                : REDONDANTE (incluse dans nb_pathologies) + PEU CORRÉLÉE (-0.0002)
   Diabetes                  : REDONDANTE (incluse dans nb_pathologies) + PEU CORRÉLÉE (-0.0152)
   Gender                    : PEU CORRÉLÉE (-0.0041)
   Handicap                  : PEU CORRÉLÉE (-0.0063)
   Hypertension              : REDONDANTE (incluse dans nb_pathologies)
   Neighbourhood             : PEU CORRÉLÉE (-0.0090)

⏳ Suppression des features non pertinentes...
 6 feature(s) supprimée(s)
  Features avant: 11 → Features après: 5

 Features finales conservées (4 features + 1 label):

   Age                       (corrélation: -0.0603)
   DaysUntilAppointement     (corrélation: +0.1863)
   Scholarship               (corrélation: +0.0292)
   nb_pathologies            (corrélation: -0.0303)



## MODÉLISATION : GRADIENT BOOSTING CLASSIFIER

In [14]:
# Séparation des features (X) et de la target (y)
print("\n=== PRÉPARATION DES DONNÉES POUR L'ENTRAÎNEMENT ===")

# Target
y = preSelectedDf['No-show']

# Features (tout sauf No-show)
X = preSelectedDf.drop(columns=['No-show'])

print(f" Target (y) : {y.name}")
print(f"  - Forme: {y.shape}")
print(f"  - Classe 0 (Présents): {(y == 0).sum()} ({(y == 0).sum()/len(y)*100:.2f}%)")
print(f"  - Classe 1 (Absents): {(y == 1).sum()} ({(y == 1).sum()/len(y)*100:.2f}%)")

print(f"\n Features (X) : {len(X.columns)} colonnes")
for col in X.columns:
    print(f"  - {col}")

print("=" * 80)


=== PRÉPARATION DES DONNÉES POUR L'ENTRAÎNEMENT ===
 Target (y) : No-show
  - Forme: (110521,)
  - Classe 0 (Présents): 88207 (79.81%)
  - Classe 1 (Absents): 22314 (20.19%)

 Features (X) : 4 colonnes
  - Scholarship
  - Age
  - DaysUntilAppointement
  - nb_pathologies


In [15]:
# Définition des features catégorielles et numériques
print("\n=== DÉFINITION DES TYPES DE FEATURES ===")

# Récupérer automatiquement les features restantes après nettoyage
all_features = [col for col in preSelectedDf.columns if col != 'No-show']

# Features catégorielles (types object ou colonnes binaires/discrètes)
categorical_features = []
numerical_features = []

for col in all_features:
    if preSelectedDf[col].dtype in ['object', 'category'] or preSelectedDf[col].nunique() <= 10:
        categorical_features.append(col)
    else:
        numerical_features.append(col)

print(f" Features catégorielles ({len(categorical_features)}): {categorical_features}")
print(f" Features numériques ({len(numerical_features)}): {numerical_features}")
print(f"\n Note: Diabetes, Hypertension, Alcoholism ont été retirées (redondantes avec nb_pathologies)")

print("=" * 80)


=== DÉFINITION DES TYPES DE FEATURES ===
 Features catégorielles (2): ['Scholarship', 'nb_pathologies']
 Features numériques (2): ['Age', 'DaysUntilAppointement']

 Note: Diabetes, Hypertension, Alcoholism ont été retirées (redondantes avec nb_pathologies)


In [16]:
# Création des transformers pour le preprocessing
print("\n=== CRÉATION DU PIPELINE DE PREPROCESSING ===")

# Transformer pour les features catégorielles
categorical_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

# Transformer pour les features numériques
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# ColumnTransformer pour appliquer les transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ])

print(" Pipeline de preprocessing créé:")
print("  - Features catégorielles → OrdinalEncoder")
print("  - Features numériques → StandardScaler")

print("=" * 80)


=== CRÉATION DU PIPELINE DE PREPROCESSING ===
 Pipeline de preprocessing créé:
  - Features catégorielles → OrdinalEncoder
  - Features numériques → StandardScaler


In [17]:
# Split train/test avec stratification
print("\n=== SPLIT TRAIN/TEST ===")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y  # Important : maintenir la proportion 80/20 dans train et test
)

print(f" Split effectué avec stratification")
print(f"\n  Train set: {len(X_train)} échantillons")
print(f"    - Classe 0 (Présents): {(y_train == 0).sum()} ({(y_train == 0).sum()/len(y_train)*100:.2f}%)")
print(f"    - Classe 1 (Absents): {(y_train == 1).sum()} ({(y_train == 1).sum()/len(y_train)*100:.2f}%)")

print(f"\n  Test set: {len(X_test)} échantillons")
print(f"    - Classe 0 (Présents): {(y_test == 0).sum()} ({(y_test == 0).sum()/len(y_test)*100:.2f}%)")
print(f"    - Classe 1 (Absents): {(y_test == 1).sum()} ({(y_test == 1).sum()/len(y_test)*100:.2f}%)")

print("=" * 80)


=== SPLIT TRAIN/TEST ===
 Split effectué avec stratification

  Train set: 88416 échantillons
    - Classe 0 (Présents): 70565 (79.81%)
    - Classe 1 (Absents): 17851 (20.19%)

  Test set: 22105 échantillons
    - Classe 0 (Présents): 17642 (79.81%)
    - Classe 1 (Absents): 4463 (20.19%)


## OPTIMISATION DES HYPERPARAMÈTRES PAR GRIDSEARCHCV

Avant d'entraîner le modèle, nous allons utiliser **GridSearchCV** pour trouver les meilleurs hyperparamètres du Gradient Boosting.

In [18]:
# Configuration du GridSearchCV pour trouver les meilleurs hyperparamètres
print("\n=== OPTIMISATION DES HYPERPARAMÈTRES (GRIDSEARCHCV) ===")
print("=" * 80)

# Création d'un modèle de base pour GridSearch
gb_base = GradientBoostingClassifier(random_state=42, verbose=0)

# Pipeline pour GridSearch
pipeline_search = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', gb_base)
])

# Grille de paramètres à tester
param_grid = {
    'classifier__n_estimators': [200, 300],
    'classifier__learning_rate': [0.05, 0.1],
    'classifier__max_depth': [5, 7],
    'classifier__min_samples_split': [10, 20],
    'classifier__min_samples_leaf': [5],
    'classifier__subsample': [0.8]
}

# Calcul du nombre de combinaisons
n_combinations = 1
for param_values in param_grid.values():
    n_combinations *= len(param_values)

print(f"\n Grille de recherche configurée:")
print(f"  - n_estimators: {param_grid['classifier__n_estimators']}")
print(f"  - learning_rate: {param_grid['classifier__learning_rate']}")
print(f"  - max_depth: {param_grid['classifier__max_depth']}")
print(f"  - min_samples_split: {param_grid['classifier__min_samples_split']}")
print(f"  - min_samples_leaf: {param_grid['classifier__min_samples_leaf']}")
print(f"  - subsample: {param_grid['classifier__subsample']}")
print(f"\n  Total de combinaisons: {n_combinations}")
print(f"  Cross-validation: 3 folds")
print(f"  Total d'entraînements: {n_combinations * 3} modèles")

# Configuration de GridSearchCV avec F1-Score comme métrique d'optimisation
grid_search = GridSearchCV(
    estimator=pipeline_search,
    param_grid=param_grid,
    cv=3,  # 3-fold cross-validation
    scoring='f1',  # Optimiser le F1-Score (classe 1 par défaut car c'est la positive)
    n_jobs=-1,  # Utiliser tous les CPU disponibles
    verbose=2
)

print(f"\n⏳ Lancement de la recherche GridSearch...")
print(f"  Métrique d'optimisation: F1-Score (classe 1 - absents)")
print(f"    Ceci peut prendre plusieurs minutes...\n")

start_time = time.time()

# Entraînement GridSearch
grid_search.fit(X_train, y_train)

elapsed_time = time.time() - start_time

print(f"\n GridSearch terminé en {elapsed_time:.2f} secondes ({elapsed_time/60:.1f} minutes)")

# Affichage des meilleurs paramètres
print(f"\n MEILLEURS HYPERPARAMÈTRES TROUVÉS:")
print(f"{'' * 80}")
best_params = grid_search.best_params_
for param_name, param_value in best_params.items():
    clean_name = param_name.replace('classifier__', '')
    print(f"  {clean_name:25s}: {param_value}")

print(f"\n Meilleur F1-Score (CV): {grid_search.best_score_:.4f}")

print("\n" + "=" * 80)


=== OPTIMISATION DES HYPERPARAMÈTRES (GRIDSEARCHCV) ===

 Grille de recherche configurée:
  - n_estimators: [200, 300]
  - learning_rate: [0.05, 0.1]
  - max_depth: [5, 7]
  - min_samples_split: [10, 20]
  - min_samples_leaf: [5]
  - subsample: [0.8]

  Total de combinaisons: 16
  Cross-validation: 3 folds
  Total d'entraînements: 48 modèles

⏳ Lancement de la recherche GridSearch...
  Métrique d'optimisation: F1-Score (classe 1 - absents)
    Ceci peut prendre plusieurs minutes...



Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END classifier__learning_rate=0.05, classifier__max_depth=5, classifier__min_samples_leaf=5, classifier__min_samples_split=20, classifier__n_estimators=200, classifier__subsample=0.8; total time=  13.9s
[CV] END classifier__learning_rate=0.05, classifier__max_depth=5, classifier__min_samples_leaf=5, classifier__min_samples_split=10, classifier__n_estimators=200, classifier__subsample=0.8; total time=  14.3s
[CV] END classifier__learning_rate=0.05, classifier__max_depth=5, classifier__min_samples_leaf=5, classifier__min_samples_split=10, classifier__n_estimators=200, classifier__subsample=0.8; total time=  14.2s
[CV] END classifier__learning_rate=0.05, classifier__max_depth=5, classifier__min_samples_leaf=5, classifier__min_samples_split=20, classifier__n_estimators=200, classifier__subsample=0.8; total time=  16.1s
[CV] END classifier__learning_rate=0.05, classifier__max_depth=5, classifier__min_samples_leaf=5, classifie

In [19]:
# Récupération du meilleur modèle et entraînement final
print("\n=== ENTRAÎNEMENT DU MODÈLE OPTIMISÉ ===")
print("=" * 80)

# Le modèle est déjà entraîné via GridSearch sur X_train
# On récupère le meilleur estimateur (pipeline complet)
model_pipeline = grid_search.best_estimator_

print(f" Meilleur modèle récupéré depuis GridSearch")
print(f"  Le modèle a déjà été entraîné avec les meilleurs hyperparamètres")

# Extraction des hyperparamètres du modèle optimisé pour affichage
gb_optimized = model_pipeline.named_steps['classifier']
print(f"\n Configuration finale du modèle:")
print(f"  - n_estimators: {gb_optimized.n_estimators}")
print(f"  - learning_rate: {gb_optimized.learning_rate}")
print(f"  - max_depth: {gb_optimized.max_depth}")
print(f"  - min_samples_split: {gb_optimized.min_samples_split}")
print(f"  - min_samples_leaf: {gb_optimized.min_samples_leaf}")
print(f"  - subsample: {gb_optimized.subsample}")

print("\n" + "=" * 80)


=== ENTRAÎNEMENT DU MODÈLE OPTIMISÉ ===
 Meilleur modèle récupéré depuis GridSearch
  Le modèle a déjà été entraîné avec les meilleurs hyperparamètres

 Configuration finale du modèle:
  - n_estimators: 300
  - learning_rate: 0.1
  - max_depth: 7
  - min_samples_split: 10
  - min_samples_leaf: 5
  - subsample: 0.8



In [20]:
# Prédictions sur les ensembles train et test
print("\n=== PRÉDICTIONS ===")

y_pred_train = model_pipeline.predict(X_train)
y_pred_test = model_pipeline.predict(X_test)

# Probabilités de prédiction (utile pour comprendre la confiance du modèle)
y_proba_test = model_pipeline.predict_proba(X_test)

print(f" Prédictions train: {len(y_pred_train)} échantillons")
print(f" Prédictions test: {len(y_pred_test)} échantillons")

print("\n" + "=" * 80)


=== PRÉDICTIONS ===
 Prédictions train: 88416 échantillons
 Prédictions test: 22105 échantillons



## ÉVALUATION DU MODÈLE OPTIMISÉ

In [21]:
# ÉVALUATION COMPLÈTE DU MODÈLE OPTIMISÉ
print("\n" + "=" * 80)
print(" ÉVALUATION COMPLÈTE DU MODÈLE OPTIMISÉ")
print("=" * 80)

# Accuracy
train_accuracy_opt = accuracy_score(y_train, y_pred_train)
test_accuracy_opt = accuracy_score(y_test, y_pred_test)

print("\n### 1⃣ ACCURACY (Exactitude globale)")
print(f"{'' * 80}")
print(f"  Train : {train_accuracy_opt:.4f} ({train_accuracy_opt*100:.2f}%)")
print(f"  Test  : {test_accuracy_opt:.4f} ({test_accuracy_opt*100:.2f}%)")
print(f"\n   Interprétation: Sur 100 prédictions, {test_accuracy_opt*100:.1f} sont correctes")

# Matrice de confusion
cm_test = confusion_matrix(y_test, y_pred_test)
vn, fp, fn, vp = cm_test.ravel()

print("\n### 2⃣ MATRICE DE CONFUSION (Test)")
print(f"{'' * 80}")
print(f"\n                      Prédit: Présent (0)    Prédit: Absent (1)")
print(f"  Réel: Présent (0)         {vn:>6}                 {fp:>6}     ")
print(f"  Réel: Absent  (1)         {fn:>6}                 {vp:>6}     ")
print(f"\n   Vrais Négatifs (VN): {vn} - Prédits présents et sont venus")
print(f"   Vrais Positifs (VP): {vp} - Prédits absents et ne sont pas venus")
print(f"   Faux Positifs (FP): {fp} - Prédits absents mais sont venus (FAUSSE ALERTE)")
print(f"   Faux Négatifs (FN): {fn} - Prédits présents mais ne sont pas venus (MANQUÉ)")

# Métriques détaillées par classe
print("\n### 3⃣ MÉTRIQUES DÉTAILLÉES PAR CLASSE")
print(f"{'' * 80}")

# Classe 0 (Présents)
precision_0 = precision_score(y_test, y_pred_test, pos_label=0)
recall_0 = recall_score(y_test, y_pred_test, pos_label=0)
f1_0 = f1_score(y_test, y_pred_test, pos_label=0)

print(f"\n CLASSE 0 (Présents - No-show=0)")
print(f"  Precision : {precision_0:.4f} ({precision_0*100:.2f}%)")
print(f"  Recall    : {recall_0:.4f} ({recall_0*100:.2f}%)")
print(f"  F1-Score  : {f1_0:.4f}")

# Classe 1 (Absents) - LA PLUS IMPORTANTE
precision_1_opt = precision_score(y_test, y_pred_test, pos_label=1)
recall_1_opt = recall_score(y_test, y_pred_test, pos_label=1)
f1_1_opt = f1_score(y_test, y_pred_test, pos_label=1)

print(f"\n CLASSE 1 (Absents - No-show=1)  PRIORITAIRE")
print(f"  Precision : {precision_1_opt:.4f} ({precision_1_opt*100:.2f}%)")
print(f"    → Sur 100 prédits 'absents', {precision_1_opt*100:.0f} le sont vraiment")
print(f"    → Fausses alertes: {(1-precision_1_opt)*100:.0f}%")
print(f"  Recall    : {recall_1_opt:.4f} ({recall_1_opt*100:.2f}%)")
print(f"    → Sur 100 absents réels, on en détecte {recall_1_opt*100:.0f}")
print(f"    → Absents manqués: {(1-recall_1_opt)*100:.0f}%")
print(f"  F1-Score  : {f1_1_opt:.4f}")

# Rapport de classification complet
print("\n### 4⃣ RAPPORT DE CLASSIFICATION COMPLET (Test)")
print(f"{'' * 80}")
print(classification_report(y_test, y_pred_test, 
                          target_names=['Classe 0: Présent', 'Classe 1: Absent']))

print("\n" + "=" * 80)


 ÉVALUATION COMPLÈTE DU MODÈLE OPTIMISÉ

### 1⃣ ACCURACY (Exactitude globale)

  Train : 0.8115 (81.15%)
  Test  : 0.7917 (79.17%)

   Interprétation: Sur 100 prédictions, 79.2 sont correctes

### 2⃣ MATRICE DE CONFUSION (Test)


                      Prédit: Présent (0)    Prédit: Absent (1)
  Réel: Présent (0)          17259                    383     
  Réel: Absent  (1)           4221                    242     

   Vrais Négatifs (VN): 17259 - Prédits présents et sont venus
   Vrais Positifs (VP): 242 - Prédits absents et ne sont pas venus
   Faux Positifs (FP): 383 - Prédits absents mais sont venus (FAUSSE ALERTE)
   Faux Négatifs (FN): 4221 - Prédits présents mais ne sont pas venus (MANQUÉ)

### 3⃣ MÉTRIQUES DÉTAILLÉES PAR CLASSE


 CLASSE 0 (Présents - No-show=0)
  Precision : 0.8035 (80.35%)
  Recall    : 0.9783 (97.83%)
  F1-Score  : 0.8823

 CLASSE 1 (Absents - No-show=1)  PRIORITAIRE
  Precision : 0.3872 (38.72%)
    → Sur 100 prédits 'absents', 39 le sont vraiment
    → F

## AMÉLIORATION DU RECALL

Le modèle optimisé a un Recall faible (~5-10%) sur la classe minoritaire (absents).
Nous allons tester plusieurs techniques pour l'améliorer :

1. **Ajustement du seuil de décision** (threshold tuning)
2. **Rééquilibrage avec SMOTE**

### Ajustement du seuil de décision

Au lieu d'utiliser le seuil par défaut de 0.5, nous allons tester différents seuils pour favoriser la détection des absents.

In [22]:
# Test de différents seuils de décision
print("\n=== AJUSTEMENT DU SEUIL DE DÉCISION ===")
print("=" * 80)

# Obtenir les probabilités de prédiction
y_proba_test_opt = model_pipeline.predict_proba(X_test)[:, 1]  # Probabilité classe 1 (absent)

# Tester différents seuils
thresholds = [0.5, 0.4, 0.3, 0.25, 0.2, 0.15]
results_threshold = []

print("\n IMPACT DU SEUIL SUR LES MÉTRIQUES:")
print(f"{'' * 80}\n")
print(f"{'Seuil':<8} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Accuracy':<12}")
print(f"{'' * 80}")

for threshold in thresholds:
    # Prédictions avec le nouveau seuil
    y_pred_threshold = (y_proba_test_opt >= threshold).astype(int)
    
    # Calcul des métriques
    precision = precision_score(y_test, y_pred_threshold, pos_label=1)
    recall = recall_score(y_test, y_pred_threshold, pos_label=1)
    f1 = f1_score(y_test, y_pred_threshold, pos_label=1)
    accuracy = accuracy_score(y_test, y_pred_threshold)
    
    results_threshold.append({
        'threshold': threshold,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy
    })
    
    print(f"{threshold:<8.2f} {precision:<12.4f} {recall:<12.4f} {f1:<12.4f} {accuracy:<12.4f}")

print(f"{'' * 80}")

# Trouver le meilleur seuil pour Recall
best_recall_idx = max(range(len(results_threshold)), key=lambda i: results_threshold[i]['recall'])
best_recall_result = results_threshold[best_recall_idx]

print(f"\n MEILLEUR SEUIL POUR RECALL:")
print(f"  Seuil optimal: {best_recall_result['threshold']:.2f}")
print(f"  Precision: {best_recall_result['precision']:.4f}")
print(f"  Recall: {best_recall_result['recall']:.4f}")
print(f"  F1-Score: {best_recall_result['f1']:.4f}")
print(f"  Accuracy: {best_recall_result['accuracy']:.4f}")

print("\n" + "=" * 80)


=== AJUSTEMENT DU SEUIL DE DÉCISION ===

 IMPACT DU SEUIL SUR LES MÉTRIQUES:


Seuil    Precision    Recall       F1-Score     Accuracy    

0.50     0.3872       0.0542       0.0951       0.7917      
0.40     0.3735       0.1714       0.2350       0.7747      
0.30     0.3416       0.4481       0.3877       0.7142      
0.25     0.3228       0.6003       0.4199       0.6651      
0.20     0.3058       0.7804       0.4394       0.5980      
0.15     0.2899       0.8824       0.4364       0.5399      


 MEILLEUR SEUIL POUR RECALL:
  Seuil optimal: 0.15
  Precision: 0.2899
  Recall: 0.8824
  F1-Score: 0.4364
  Accuracy: 0.5399



### Rééquilibrage avec SMOTE

SMOTE (Synthetic Minority Over-sampling Technique) génère des exemples synthétiques de la classe minoritaire pour rééquilibrer le dataset.

In [23]:
# Application de SMOTE sur les données d'entraînement
from imblearn.over_sampling import SMOTE

print("\n=== RÉÉQUILIBRAGE AVEC SMOTE ===")
print("=" * 80)

# Distribution avant SMOTE
print("\n Distribution AVANT SMOTE:")
print(f"  Classe 0 (Présents): {(y_train == 0).sum()} ({(y_train == 0).sum()/len(y_train)*100:.2f}%)")
print(f"  Classe 1 (Absents): {(y_train == 1).sum()} ({(y_train == 1).sum()/len(y_train)*100:.2f}%)")
print(f"  Total: {len(y_train)}")

# Appliquer le preprocessing avant SMOTE
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Application de SMOTE
smote = SMOTE(random_state=42, sampling_strategy=0.5)  # Rééquilibre à 50% (1:2 au lieu de 1:4)
X_train_smote, y_train_smote = smote.fit_resample(X_train_preprocessed, y_train)

# Distribution après SMOTE
print("\n Distribution APRÈS SMOTE:")
print(f"  Classe 0 (Présents): {(y_train_smote == 0).sum()} ({(y_train_smote == 0).sum()/len(y_train_smote)*100:.2f}%)")
print(f"  Classe 1 (Absents): {(y_train_smote == 1).sum()} ({(y_train_smote == 1).sum()/len(y_train_smote)*100:.2f}%)")
print(f"  Total: {len(y_train_smote)} (augmentation de {len(y_train_smote) - len(y_train)} exemples)")

print("\n⏳ Entraînement du modèle avec données SMOTE...")
start_time = time.time()

# Créer et entraîner un nouveau modèle avec les meilleurs hyperparamètres
# Extraire les paramètres sans le préfixe 'classifier__'
gb_params = {k.replace('classifier__', ''): v for k, v in best_params.items()}
gb_smote = GradientBoostingClassifier(**gb_params)
gb_smote.fit(X_train_smote, y_train_smote)

elapsed_time = time.time() - start_time
print(f"Entraînement terminé en {elapsed_time:.2f} secondes")

# Prédictions
y_pred_test_smote = gb_smote.predict(X_test_preprocessed)

# Évaluation
print("\n RÉSULTATS AVEC SMOTE:")
print(f"{'' * 80}")

precision_smote = precision_score(y_test, y_pred_test_smote, pos_label=1)
recall_smote = recall_score(y_test, y_pred_test_smote, pos_label=1)
f1_smote = f1_score(y_test, y_pred_test_smote, pos_label=1)
accuracy_smote = accuracy_score(y_test, y_pred_test_smote)

print(f"  Precision (classe 1): {precision_smote:.4f} ({precision_smote*100:.2f}%)")
print(f"  Recall (classe 1):    {recall_smote:.4f} ({recall_smote*100:.2f}%)")
print(f"  F1-Score (classe 1):  {f1_smote:.4f}")
print(f"  Accuracy:             {accuracy_smote:.4f} ({accuracy_smote*100:.2f}%)")

# Comparaison avec le modèle optimisé sans SMOTE
print(f"\n COMPARAISON SMOTE vs MODÈLE OPTIMISÉ:")
print(f"{'' * 80}")
print(f"  Recall: {recall_1_opt:.4f} → {recall_smote:.4f} (gain: {(recall_smote - recall_1_opt)*100:+.2f}%)")
print(f"  F1-Score: {f1_1_opt:.4f} → {f1_smote:.4f} (gain: {(f1_smote - f1_1_opt)*100:+.2f}%)")

print("\n" + "=" * 80)


=== RÉÉQUILIBRAGE AVEC SMOTE ===

 Distribution AVANT SMOTE:
  Classe 0 (Présents): 70565 (79.81%)
  Classe 1 (Absents): 17851 (20.19%)
  Total: 88416

 Distribution APRÈS SMOTE:
  Classe 0 (Présents): 70565 (66.67%)
  Classe 1 (Absents): 35282 (33.33%)
  Total: 105847 (augmentation de 17431 exemples)

⏳ Entraînement du modèle avec données SMOTE...


/home/pthaochanta/CODE/PET/ai/ai-dev-alyra-complete-training/livrables/ml/venv/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Entraînement terminé en 18.48 secondes

 RÉSULTATS AVEC SMOTE:

  Precision (classe 1): 0.3671 (36.71%)
  Recall (classe 1):    0.2283 (22.83%)
  F1-Score (classe 1):  0.2815
  Accuracy:             0.7647 (76.47%)

 COMPARAISON SMOTE vs MODÈLE OPTIMISÉ:

  Recall: 0.0542 → 0.2283 (gain: +17.41%)
  F1-Score: 0.0951 → 0.2815 (gain: +18.64%)



In [24]:
print("\n" + "=" * 80)
print("=== TECHNIQUE COMBINÉE: AJUSTEMENT SEUIL + GB + SMOTE ===")
print("=" * 80)

# Obtenir les probabilités du modèle GB+SMOTE
y_proba_smote = gb_smote.predict_proba(X_test_preprocessed)[:, 1]

# Test de différents seuils
thresholds_to_test = [0.5, 0.4, 0.3, 0.25, 0.2, 0.15]
results_threshold_smote = []

print("\n TEST DE DIFFÉRENTS SEUILS SUR LE MODÈLE GB+SMOTE:")
print(f"{'' * 80}")
print(f"{'Seuil':<10} {'Precision':<15} {'Recall':<15} {'F1-Score':<15} {'Accuracy':<15}")
print(f"{'' * 80}")

for threshold in thresholds_to_test:
    y_pred_threshold = (y_proba_smote >= threshold).astype(int)
    
    precision = precision_score(y_test, y_pred_threshold, pos_label=1)
    recall = recall_score(y_test, y_pred_threshold, pos_label=1)
    f1 = f1_score(y_test, y_pred_threshold, pos_label=1)
    accuracy = accuracy_score(y_test, y_pred_threshold)
    
    results_threshold_smote.append({
        'threshold': threshold,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy
    })
    
    print(f"{threshold:<10.2f} {precision:<15.4f} {recall:<15.4f} {f1:<15.4f} {accuracy:<15.4f}")

print(f"{'' * 80}")

# Trouver le meilleur seuil pour le recall
best_recall_threshold = max(results_threshold_smote, key=lambda x: x['recall'])
print(f"\n MEILLEUR SEUIL POUR LE RECALL:")
print(f"  Seuil optimal: {best_recall_threshold['threshold']}")
print(f"  Precision: {best_recall_threshold['precision']:.4f} ({best_recall_threshold['precision']*100:.2f}%)")
print(f"  Recall: {best_recall_threshold['recall']:.4f} ({best_recall_threshold['recall']*100:.2f}%)")
print(f"  F1-Score: {best_recall_threshold['f1']:.4f}")
print(f"  Accuracy: {best_recall_threshold['accuracy']:.4f} ({best_recall_threshold['accuracy']*100:.2f}%)")

# Trouver le meilleur seuil pour le F1-Score
best_f1_threshold = max(results_threshold_smote, key=lambda x: x['f1'])
print(f"\n MEILLEUR SEUIL POUR LE F1-SCORE:")
print(f"  Seuil optimal: {best_f1_threshold['threshold']}")
print(f"  Precision: {best_f1_threshold['precision']:.4f} ({best_f1_threshold['precision']*100:.2f}%)")
print(f"  Recall: {best_f1_threshold['recall']:.4f} ({best_f1_threshold['recall']*100:.2f}%)")
print(f"  F1-Score: {best_f1_threshold['f1']:.4f}")
print(f"  Accuracy: {best_f1_threshold['accuracy']:.4f} ({best_f1_threshold['accuracy']*100:.2f}%)")

# Comparaison avec GB+SMOTE seul (seuil par défaut 0.5)
print(f"\n COMPARAISON GB+SMOTE vs GB+SMOTE+SEUIL:")
print(f"{'' * 80}")
print(f"  Recall: {recall_smote:.4f} → {best_recall_threshold['recall']:.4f} (gain: {(best_recall_threshold['recall'] - recall_smote)*100:+.2f}%)")
print(f"  F1-Score: {f1_smote:.4f} → {best_f1_threshold['f1']:.4f} (gain: {(best_f1_threshold['f1'] - f1_smote)*100:+.2f}%)")
print(f"  Precision: {precision_smote:.4f} → {best_recall_threshold['precision']:.4f} (variation: {(best_recall_threshold['precision'] - precision_smote)*100:+.2f}%)")

# Matrice de confusion pour le meilleur seuil (recall optimisé)
print(f"\n MATRICE DE CONFUSION AVEC SEUIL OPTIMAL ({best_recall_threshold['threshold']}):")
print(f"{'' * 80}")

# Générer les prédictions avec le seuil optimal
y_pred_optimal = (y_proba_smote >= best_recall_threshold['threshold']).astype(int)

# Calculer la matrice de confusion
cm_optimal = confusion_matrix(y_test, y_pred_optimal)
vn_opt, fp_opt, fn_opt, vp_opt = cm_optimal.ravel()

print(f"\n                      Prédit: Présent (0)    Prédit: Absent (1)")
print(f"  Réel: Présent (0)         {vn_opt:>6}                 {fp_opt:>6}     ")
print(f"  Réel: Absent  (1)         {fn_opt:>6}                 {vp_opt:>6}     ")

print(f"\n  Interprétation:")
print(f"    Vrais Négatifs (VN): {vn_opt} - Correctement prédits présents")
print(f"    Vrais Positifs (VP): {vp_opt} - Correctement prédits absents")
print(f"    Faux Positifs (FP): {fp_opt} - Prédits absents mais venus (coût: rappel inutile)")
print(f"    Faux Négatifs (FN): {fn_opt} - Prédits présents mais absents (coût: rdv perdu)")

print(f"\n  Taux de détection des absents: {vp_opt}/{vp_opt + fn_opt} = {best_recall_threshold['recall']*100:.2f}%")
print(f"  Taux de fausses alertes: {fp_opt}/{vn_opt + fp_opt} = {fp_opt/(vn_opt + fp_opt)*100:.2f}%")

print("\n" + "=" * 80)


=== TECHNIQUE COMBINÉE: AJUSTEMENT SEUIL + GB + SMOTE ===

 TEST DE DIFFÉRENTS SEUILS SUR LE MODÈLE GB+SMOTE:

Seuil      Precision       Recall          F1-Score        Accuracy       

0.50       0.3671          0.2283          0.2815          0.7647         
0.40       0.3328          0.4988          0.3992          0.6969         
0.30       0.3037          0.7578          0.4336          0.6004         
0.25       0.2955          0.8400          0.4372          0.5633         
0.20       0.2853          0.8976          0.4330          0.5254         
0.15       0.2739          0.9352          0.4237          0.4864         


 MEILLEUR SEUIL POUR LE RECALL:
  Seuil optimal: 0.15
  Precision: 0.2739 (27.39%)
  Recall: 0.9352 (93.52%)
  F1-Score: 0.4237
  Accuracy: 0.4864 (48.64%)

 MEILLEUR SEUIL POUR LE F1-SCORE:
  Seuil optimal: 0.25
  Precision: 0.2955 (29.55%)
  Recall: 0.8400 (84.00%)
  F1-Score: 0.4372
  Accuracy: 0.5633 (56.33%)

 COMPARAISON GB+SMOTE vs GB+SMOTE+SEUIL:

  

##  CONCLUSION

Ce projet a permis d'optimiser un modèle **Gradient Boosting** pour prédire les absences aux rendez-vous médicaux. 

**Résultats clés:**
- GridSearchCV a optimisé les hyperparamètres (learning_rate, max_depth, n_estimators)
- SMOTE a rééquilibré le dataset (80/20 → 67/33)
- L'ajustement du seuil de décision a maximisé le Recall
- La combinaison des 3 techniques améliore significativement la détection des absents

**Recommandation:** Utiliser le modèle GB+SMOTE avec un seuil ajusté entre 0.15 et 0.25 selon le compromis souhaité entre Recall (détection) et Precision (fausses alertes).